In [1]:
import isort
import sys
import platform
import importlib
from types import ModuleType

In [2]:
# module_names = isort.stdlibs.py38.stdlib

In [3]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = set()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.add(name)
    return external_packages

In [4]:
def task1():
    external_packages = get_stdlib_packages()
    v = sys.version_info
    print(f'Python {v.major}.{v.minor}.{v.micro} on {platform.system()} {platform.release()}')
    print(f'Stdlib contains {len(external_packages)} external modules and packages: ')
    print(external_packages)

In [5]:
task1()

Python 3.8.10 on Linux 5.15.0-50-generic
Stdlib contains 212 external modules and packages: 
{'stat', 'inspect', 'pyclbr', 'pickle', 'dummy_threading', 'queue', 'timeit', 'pipes', 'turtle', 'smtplib', 'textwrap', 'time', 'sndhdr', 'runpy', 'sunau', 'crypt', 'cmath', 'sqlite3', 'turtledemo', 'csv', 'functools', 'weakref', 'unicodedata', 'wave', 'zipimport', 'curses', 'ftplib', 'msilib', 'resource', 'encodings', 'io', 'tempfile', 'tokenize', 'quopri', 'types', 'bisect', 'imp', 'subprocess', 'urllib', 'winreg', 'xml', 'gc', 'enum', 'chunk', 'zipfile', 'pickletools', 'datetime', 'filecmp', 'ast', 'zipapp', 'dataclasses', 'statistics', 'errno', 'pstats', 'netrc', 'webbrowser', 'posixpath', 'tty', 'plistlib', 'tabnanny', 'contextlib', 'sre', 'bdb', 'dbm', 'argparse', 'select', 'site', 'copy', 'math', 'hmac', 'sys', 'uu', 'codeop', 'posix', 'copyreg', 'modulefinder', 'mailcap', 'nis', 'glob', 'signal', 'readline', 'fractions', 'multiprocessing', 'mimetypes', 'spwd', 'tkinter', 'contextvars', 

In [6]:
def get_real_packages(package_names):
    real_modules = set()
    not_importable_modules = set()
    for name in package_names:
        try:
            importlib.import_module(name)
            real_modules.add(name)
            if name != 'importlib':
                del sys.modules[name]
        except:
            not_importable_modules.add(name)
    return real_modules, not_importable_modules

In [7]:
def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [8]:
def task2():
    v = sys.version_info
    real_modules, not_importable_modules = get_real()
    print(f'These StdLib packages on Python-{v.major}.{v.minor}.{v.micro}/{platform.system()} {platform.release()} '
          f'are not importable:')
    print(not_importable_modules)

    # for name in real_modules:
    #     del sys.modules[name]

In [9]:
task2()

These StdLib packages on Python-3.8.10/Linux 5.15.0-50-generic are not importable:
{'cgitb', 'smtpd', 'turtle', 'sre', 'unittest', 'turtledemo', 'imaplib', 'msilib', 'tarfile', 'ensurepip', 'winreg', 'doctest', 'winsound', 'tkinter', 'formatter', 'poplib', 'msvcrt', 'shelve', 'mailbox', 'optparse'}


In [10]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = set()

    try:
        importlib.import_module(name)
    except:
        print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            dp_names.add(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    # del sys.modules[name]

    return dp_names

In [11]:
def core_modules(real_modules):
    # real_modules, _ = get_real()
    core_module_names = set()
    for r_name in real_modules:
        dp_names = module_dependency(real_modules, r_name)
        if len(dp_names) == 0:
            core_module_names.add(r_name)

    return core_module_names


In [12]:
def most_dependent_modules(real_modules):
    dp_dict = dict((name, 0) for name in real_modules)
    for md_name in real_modules:
        count = len(module_dependency(real_modules, md_name))
        dp_dict[md_name] = count

    sorted_dp_dict = dict(sorted(dp_dict.items(), key=lambda item: item[1], reverse=True))
    i = 0
    most_dependent_module_names = dict()
    for k, v in sorted_dp_dict.items():
        if i > 5:
            break
        most_dependent_module_names[k] = v
        i = i + 1

    return most_dependent_module_names



In [13]:
def task3():
    real_modules, not_importable_modules = get_real()

    # re-import libs:
    for r in real_modules:
        pass

    # print(f'real modules: {real_modules}')
    core_module_names = core_modules(real_modules)

    dp_dict = most_dependent_modules(real_modules)

    print(f'The following StdLib packages are most dependent:')
    for k, v in dp_dict.items():
        print(f'{k}: {v}')
    print(f'The {len(core_module_names)} core packages are:')
    print(core_module_names)

    # for name in real_modules:
    #     del sys.modules[name]

In [14]:
task3()

err name: inspect
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_202597/309355583.py", line 1, in <cell line: 1>
    task3()
  File "/tmp/ipykernel_202597/830394055.py", line 4, in task3
    core_module_names = core_modules(real_modules)
  File "/tmp/ipykernel_202597/3138469120.py", line 5, in core_modules
    dp_names = module_dependency(real_modules, r_name)
  File "/tmp/ipykernel_202597/758718900.py", line 10, in module_dependency
    for key, val in vars(sys.modules[name]).items():
KeyError: 'inspect'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/linhnm/.local/lib/python3.8/site-packages/executing/executing.py", line 312, in executing
    args = executing_cache[key]
KeyError: (<code object run_code at 0x7f663be71500, file "/home/linhnm/.local/lib/pytho

In [ ]:
def explore_package(package_name):
    importlib.import_module(package_name)
    pack = sys.modules[package_name]
    file_name = 'builtin_binary'
    try:
        file_name = pack.__file__
        n = len(file_name)
        # print(file_name[n-3:n])
        if file_name[n-3:n] == '.py':
            # print(file_name)
            return package_name, file_name, 0
        elif file_name[n-3:n] == '.so':
            # print(file_name)
            return package_name, file_name, 1
        else:
            raise Exception(f'unsupported file: {file_name}')
    except:
        return package_name, file_name, 2

p, f, r = explore_package('locale')
print(r)

In [ ]:
real_modules, _ = get_real()

for md in real_modules:
    p, f, r = explore_package(md)
    print(f'package name: {p}, file: {f}, type: {r}')

In [ ]:
import inspect
import fileinput

import sys

real_modules, _ = get_real()

print(real_modules)

# for r in real_modules:
#     importlib.import_module(r)
#
# # importlib.import_module('fileinput')
# for name, obj in inspect.getmembers(fileinput):
#         if inspect.isclass(obj):
#             print(obj)